In [18]:
!pip install -q langchain openai faiss-cpu tiktoken langchain_community


In [19]:
import os
os.environ["OPENAI_API_KEY"] = "XXX"  # 🔐 Replace with your OpenAI API Key


In [20]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader

#  Load sample document (replace with your own file)
with open("rag_context.txt", "w") as f:
    f.write("""Introduction
Life skills are essential tools for managing everyday responsibilities, solving problems, and staying organized. By combining practical life planning systems with basic preparedness, anyone can increase productivity, reduce stress, and navigate life more confidently.

1. Travel Planning Essentials
Efficient travel planning saves time, money, and hassle.

Use checklists: Create a packing checklist categorized by clothing, tech, toiletries, documents, etc.

Plan your itinerary: Use tools like Google Maps, Rome2Rio, or TripIt to organize transport and sightseeing.

Back up documents: Store digital copies of IDs, tickets, and hotel bookings in the cloud (e.g., Google Drive).

Emergency contacts: Have printed and digital access to emergency phone numbers and embassy info.

2. Creating a Personal Knowledge System
A personal knowledge system (PKS) helps capture, organize, and reuse information.

Use note-taking apps: Tools like Notion, Obsidian, or OneNote allow you to structure your thoughts.

Tagging: Tag notes by topic or theme (e.g., #health, #finance) to enable fast retrieval.

Templates: Use repeatable templates for meeting notes, journals, or decision logs.

Link ideas: Use backlinks or cross-notes to connect thoughts across domains for deeper learning.

3. Basic First Aid Knowledge Everyone Should Know
First aid can save lives in emergencies.

Key skills: Learn how to treat minor cuts, burns, sprains, and recognize signs of a stroke or heart attack.

CPR: Take a certified CPR course, or learn the basics through reputable platforms like Red Cross or WHO.

First aid kit: Maintain a stocked kit at home and in your car—include bandages, antiseptics, gloves, and pain relievers.

Emergency response: Memorize your local emergency number and teach others in your household.

4. Weekly and Monthly Planning Systems
Structured planning prevents decision fatigue and boosts control over life.

Weekly review: Every Sunday, set goals, review last week’s wins/losses, and prep a flexible schedule.

Calendar blocking: Reserve time for work, personal errands, exercise, and downtime using Google Calendar or a paper planner.

Monthly check-in: Review budgets, renew subscriptions, schedule appointments, and check on long-term goals.

5. Financial Micro-Planning
Life planning includes micro-financial skills to avoid stress.

Track expenses: Use apps like Mint, PocketGuard, or Excel sheets.

Bill calendar: Set reminders for rent, utilities, credit cards, and subscriptions.

Emergency fund: Start by saving one month’s expenses, then build up to 3–6 months.

Subscriptions audit: Monthly check for services you no longer use (streaming, apps, software).



""")

loader = TextLoader("rag_context.txt")
documents = loader.load()

#  Split text
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

#  Embeddings + FAISS Index
embedding = OpenAIEmbeddings()
db = FAISS.from_documents(docs, embedding)



AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: XXX. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
from langchain.prompts import PromptTemplate

template = """
You are an AI assistant answering based on retrieved knowledge. Use the context to answer accurately.

Context:
{context}

Question:
{question}

Answer:"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)


In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(),
    chain_type_kwargs={"prompt": prompt},
)


In [ ]:
query = "What are essential life skills and planning systems that can help with everyday organization and productivity?"


result = qa_chain.run(query)
print("💡 Answer:", result)
